In [843]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import sklearn

In [844]:
df = pd.read_csv('data.csv')
df = df[-550:]
df

,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
473,Kevin McHale,29,BOS,0.0,254.0,780,0.326,77,39.7,26.1,9.9,2.6,0.5,2.2,0.604,0.000,0.836,14.8,0.232
474,Dominique Wilkins,27,ATL,0.0,128.0,780,0.164,79,37.6,29.0,6.3,3.3,1.5,0.6,0.463,0.292,0.818,12.2,0.197
475,Charles Barkley,23,PHI,1.0,113.0,780,0.145,68,40.3,23.0,14.6,4.9,1.8,1.5,0.594,0.202,0.761,12.0,0.210
476,Hakeem Olajuwon,24,HOU,1.0,28.0,780,0.036,75,36.8,23.4,11.4,2.9,1.9,3.4,0.508,0.200,0.702,10.9,0.189
477,Isiah Thomas,25,DET,0.0,17.0,780,0.022,81,37.2,20.6,3.9,10.0,1.9,0.2,0.463,0.194,0.768,7.6,0.121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,Stephen Curry,33,GSW,0.0,4.0,1000,0.004,64,34.5,25.5,5.2,6.3,1.3,0.4,0.437,0.380,0.923,8.0,0.173
1019,Chris Paul,36,PHO,0.0,2.0,1000,0.002,65,32.9,14.7,4.4,10.8,1.9,0.3,0.493,0.317,0.837,9.4,0.210
1020,DeMar DeRozan,32,CHI,0.0,1.0,1000,0.001,76,36.1,27.9,5.2,4.9,0.9,0.3,0.504,0.352,0.877,8.8,0.154
1021,Kevin Durant,33,BRK,0.0,1.0,1000,0.001,55,37.2,29.9,7.4,6.4,0.9,0.9,0.518,0.383,0.910,8.4,0.198


In [845]:
cur_stats = pd.read_csv('current_stats.csv')
cur_stats_adv = pd.read_csv('current_stats_adv.csv')

cur_stats['WS'] = cur_stats_adv['WS']
cur_stats['WS/48'] = cur_stats_adv['WS/48']
cur_stats = cur_stats.drop(['Tm'], axis=1)

cur_stats = cur_stats.drop([col for col in cur_stats.columns if col not in df.columns], axis=1)
cur_stats.head()

,Player,Age,G,MP,FG%,3P%,FT%,TRB,AST,STL,BLK,PTS,WS,WS/48
0,Precious Achiuwa,23,34,23.3,0.486,0.247,0.684,6.5,1.1,0.7,0.7,10.4,1.6,0.099
1,Steven Adams,29,42,27.0,0.597,0.000,0.364,11.5,2.3,0.9,1.1,8.6,3.4,0.145
2,Bam Adebayo,25,52,35.3,0.546,0.091,0.806,10.1,3.3,1.2,0.8,21.6,6.1,0.159
3,Ochai Agbaji,22,35,14.0,0.486,0.394,0.625,1.6,0.5,0.1,0.1,4.1,0.7,0.068
4,Santi Aldama,22,53,22.1,0.477,0.367,0.738,4.7,1.2,0.7,0.7,9.5,3.6,0.146


In [846]:
# Remove Player and Team and drop duplicate
df = df.drop(['Player', 'Tm'], axis=1)
df.head()

,Age,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
473,29,0.0,254.0,780,0.326,77,39.7,26.1,9.9,2.6,0.5,2.2,0.604,0.000,0.836,14.8,0.232
474,27,0.0,128.0,780,0.164,79,37.6,29.0,6.3,3.3,1.5,0.6,0.463,0.292,0.818,12.2,0.197
475,23,1.0,113.0,780,0.145,68,40.3,23.0,14.6,4.9,1.8,1.5,0.594,0.202,0.761,12.0,0.210
476,24,1.0,28.0,780,0.036,75,36.8,23.4,11.4,2.9,1.9,3.4,0.508,0.200,0.702,10.9,0.189
477,25,0.0,17.0,780,0.022,81,37.2,20.6,3.9,10.0,1.9,0.2,0.463,0.194,0.768,7.6,0.121


In [847]:
# Setting NaN features to the average

# Calculating average
df_mean = df.dropna()
mean_vals = df_mean.mean()

# Setting values
for row in range(len(df)):
    for i, val in enumerate(df.iloc[row]):
        if np.isnan(val):
            df.iat[row,i] = mean_vals[i]

df.head()

,Age,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
473,29,0.0,254.0,780,0.326,77,39.7,26.1,9.9,2.6,0.5,2.2,0.604,0.000,0.836,14.8,0.232
474,27,0.0,128.0,780,0.164,79,37.6,29.0,6.3,3.3,1.5,0.6,0.463,0.292,0.818,12.2,0.197
475,23,1.0,113.0,780,0.145,68,40.3,23.0,14.6,4.9,1.8,1.5,0.594,0.202,0.761,12.0,0.210
476,24,1.0,28.0,780,0.036,75,36.8,23.4,11.4,2.9,1.9,3.4,0.508,0.200,0.702,10.9,0.189
477,25,0.0,17.0,780,0.022,81,37.2,20.6,3.9,10.0,1.9,0.2,0.463,0.194,0.768,7.6,0.121


In [848]:
# Dropping First, Pts Won, and Pts Max. Using MVP shares for as the predicted value
df = df.drop(['First', 'Pts Won', 'Pts Max'], axis=1)
df.tail()

,Age,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
1018,33,0.004,64,34.5,25.5,5.2,6.3,1.3,0.4,0.437,0.380,0.923,8.0,0.173
1019,36,0.002,65,32.9,14.7,4.4,10.8,1.9,0.3,0.493,0.317,0.837,9.4,0.210
1020,32,0.001,76,36.1,27.9,5.2,4.9,0.9,0.3,0.504,0.352,0.877,8.8,0.154
1021,33,0.001,55,37.2,29.9,7.4,6.4,0.9,0.9,0.518,0.383,0.910,8.4,0.198
1022,37,0.001,56,37.2,30.3,8.2,6.2,1.3,1.1,0.524,0.359,0.756,7.5,0.172


In [849]:
full_df = pd.concat([cur_stats, df])
normalized_df = full_df.drop(['Player', 'Share'], axis=1)
normalized_df['WS'] = list(map(lambda x: x**2, normalized_df['WS']))

In [850]:
normalized_df = (normalized_df-normalized_df.min())/(normalized_df.max()-normalized_df.min())
normalized_df['Player'] = full_df['Player']
normalized_df['Share'] = full_df['Share']
normalized_df

,Age,G,MP,FG%,3P%,FT%,TRB,AST,STL,BLK,PTS,WS,WS/48,Player,Share
0,0.173913,0.407407,0.533181,0.486,0.247,0.684,0.347594,0.075862,0.175,0.152174,0.288089,0.005696,0.458207,Precious Achiuwa,NaN
1,0.434783,0.506173,0.617849,0.597,0.000,0.364,0.614973,0.158621,0.225,0.239130,0.238227,0.025721,0.481370,Steven Adams,NaN
2,0.260870,0.629630,0.807780,0.546,0.091,0.806,0.540107,0.227586,0.300,0.173913,0.598338,0.082792,0.488419,Bam Adebayo,NaN
3,0.130435,0.419753,0.320366,0.486,0.394,0.625,0.085561,0.034483,0.025,0.021739,0.113573,0.001090,0.442598,Ochai Agbaji,NaN
4,0.130435,0.641975,0.505721,0.477,0.367,0.738,0.251337,0.082759,0.175,0.152174,0.263158,0.028836,0.481873,Santi Aldama,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,0.608696,0.777778,0.789474,0.437,0.380,0.923,0.278075,0.434483,0.325,0.086957,0.706371,0.142399,0.495468,NaN,0.004
1019,0.739130,0.790123,0.752860,0.493,0.317,0.837,0.235294,0.744828,0.475,0.065217,0.407202,0.196600,0.514099,NaN,0.002
1020,0.565217,0.925926,0.826087,0.504,0.352,0.877,0.278075,0.337931,0.225,0.065217,0.772853,0.172303,0.485901,NaN,0.001
1021,0.608696,0.666667,0.851259,0.518,0.383,0.910,0.395722,0.441379,0.225,0.195652,0.828255,0.156995,0.508056,NaN,0.001


In [851]:
# normalized_df = normalized_df[['WS','WS/48', 'Player', 'Share']]

In [852]:
data = normalized_df.loc[normalized_df['Player'].isna()].drop(['Player'], axis=1).to_numpy()
y = data[:, -1]
data = np.delete(data, -1, axis=1)
X = data
X, y

(array([[0.43478261, 0.9382716 , 0.90846682, ..., 0.72299169, 0.48736205,
         0.52517623],
        [0.34782609, 0.96296296, 0.8604119 , ..., 0.8033241 , 0.33116768,
         0.50755287],
        [0.17391304, 0.82716049, 0.9221968 , ..., 0.63711911, 0.32039872,
         0.51409869],
        ...,
        [0.56521739, 0.92592593, 0.82608696, ..., 0.77285319, 0.17230331,
         0.48590131],
        [0.60869565, 0.66666667, 0.85125858, ..., 0.82825485, 0.15699537,
         0.50805639],
        [0.7826087 , 0.67901235, 0.85125858, ..., 0.83933518, 0.12515575,
         0.49496475]]),
 array([0.326, 0.164, 0.145, 0.036, 0.022, 0.014, 0.012, 0.005, 0.004,
        0.004, 0.003, 0.001, 0.001, 0.001, 0.831, 0.659, 0.635, 0.136,
        0.108, 0.069, 0.05 , 0.045, 0.033, 0.019, 0.005, 0.004, 0.003,
        0.001, 0.001, 0.001, 0.001, 0.782, 0.704, 0.426, 0.235, 0.211,
        0.111, 0.033, 0.026, 0.024, 0.021, 0.005, 0.005, 0.004, 0.004,
        0.004, 0.004, 0.001, 0.001, 0.001, 0.691, 0.66

In [853]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = list(map(lambda x: torch.FloatTensor(x), train_test_split(X, y, test_size=0.1, random_state=0)))
X_train, y_train

(tensor([[0.1739, 0.9877, 0.9153,  ..., 0.6343, 0.2994, 0.4950],
         [0.4348, 0.8889, 0.8009,  ..., 0.6039, 0.1354, 0.4819],
         [0.7826, 0.6790, 0.8513,  ..., 0.8393, 0.1252, 0.4950],
         ...,
         [0.6087, 0.9753, 0.8558,  ..., 0.4848, 0.1883, 0.4824],
         [0.1739, 1.0000, 0.8535,  ..., 0.5208, 0.2841, 0.4975],
         [0.2609, 0.9877, 0.9314,  ..., 0.5845, 0.2315, 0.4834]]),
 tensor([0.3370, 0.0270, 0.0010, 0.3470, 0.0010, 0.9320, 0.1490, 0.0100, 0.0840,
         0.2050, 0.0040, 0.6600, 0.3540, 0.0450, 0.0310, 0.0050, 0.0010, 0.5770,
         0.0300, 0.0160, 0.0030, 0.0080, 0.0010, 0.6270, 0.0050, 0.0010, 0.0110,
         0.0320, 0.9690, 0.0060, 0.2160, 0.0020, 0.0030, 0.0010, 0.0200, 0.0130,
         0.0020, 0.0480, 0.4490, 0.0020, 0.0160, 0.0060, 0.0260, 0.1640, 0.0150,
         0.0010, 0.8580, 0.0030, 0.4950, 0.2580, 0.4200, 0.7260, 0.0110, 0.0010,
         0.1460, 0.2860, 0.0070, 0.0010, 0.0010, 0.3480, 0.7820, 0.0740, 0.3930,
         0.0170, 0.0020, 0.

In [854]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layers
        self.m1 = nn.Linear(X.shape[1], 256)
        self.f1 = nn.LeakyReLU()
        self.m2 = nn.Linear(256, 256)
        self.f2 = nn.LeakyReLU()
        self.m3 = nn.Linear(256, 64)
        self.f3 = nn.LeakyReLU()
        self.m4 = nn.Linear(64, 1)
        self.f4 = nn.ReLU()
    
    def forward(self, x):
        x = self.m1(x)
        x = self.f1(x)
        x = self.m2(x)
        x = self.f2(x)
        x = self.m3(x)
        x = self.f3(x)
        x = self.m4(x)
        x = self.f4(x)
        return x


In [855]:
model = Network()

In [856]:
# Hyperparameters
learning_rate = 1e-9
num_epochs = 5000

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.1, weight_decay=0.9)

# Training Model
train_loss = []
epoch_durs = []
for epoch in range(num_epochs):
  output = model(X_train)
  loss = torch.sum((output - y_train)**2)
  train_loss.append(loss.item())
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(f'epoch {epoch} of {num_epochs} loss: {loss.item()}\t\t\t\t\t\t\t\t', end='\r')

plt.plot(range(len(train_loss)), train_loss)
plt.show()

In [ ]:
output = model(X_val)
output = output.detach()
val_df = pd.DataFrame(np.hstack([output, y_val.reshape(-1, 1)]),
                      columns=['Pred', 'Actual'])

y_pred = model(X_val)
y_pred = y_pred.detach().reshape(1, -1)[0]
print(sklearn.metrics.r2_score(y_val, y_pred))
print(val_df.mean())
val_df

-0.006514142886917318
Pred      0.165343
Actual    0.186655
dtype: float32


,Pred,Actual
0,0.165906,0.584
1,0.167819,0.053
2,0.165656,0.019
3,0.166892,0.006
4,0.164713,0.507
5,0.160435,0.033
6,0.165663,0.001
7,0.168435,0.069
8,0.171342,0.008
9,0.167958,0.166


In [ ]:
cur_df = normalized_df.loc[normalized_df['Share'].isna()].drop(['Share'], axis=1)
cur_val_df = pd.DataFrame(columns=['Pred'], data=model(torch.FloatTensor(cur_df.drop(['Player'], axis=1).to_numpy())).detach())
cur_val_df['Player'] = normalized_df.loc[normalized_df['Share'].isna()]['Player']
cur_val_df = cur_val_df.sort_values(by=['Pred'])
cur_val_df = cur_val_df.dropna()
cur_val_df[-50:]

,Pred,Player
25,0.165359,Desmond Bane
217,0.165485,Tyler Herro
207,0.165501,Josh Hart
266,0.165589,Tre Jones
24,0.165723,Paolo Banchero
206,0.165786,Josh Hart
505,0.165811,Jarred Vanderbilt
504,0.165966,Jarred Vanderbilt
198,0.165986,James Harden
182,0.166038,Draymond Green


In [ ]:
def calc_pct(x):
    l = list(map(lambda i: i**100, x))
    return [val/sum(l) for val in l]
    
mvp_prediction_df = cur_val_df[-10:]
input = mvp_prediction_df['Pred'].to_numpy()
mvp_prediction_df['% Chance'] = calc_pct(input)
mvp_prediction_df = mvp_prediction_df.drop(['Pred'], axis=1)
mvp_prediction_df = mvp_prediction_df.reset_index(drop=True)
mvp_prediction_df = mvp_prediction_df.sort_values(by=['% Chance'], ascending=False)
mvp_prediction_df

/var/folders/_0/zqnz262926vbc83620bb5j9w0000gn/T/ipykernel_7842/2943891854.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mvp_prediction_df['% Chance'] = calc_pct(input)


,Player,% Chance
9,Luka Dončić,0.419698
8,Nikola Jokić,0.191238
7,Giannis Antetokounmpo,0.088235
6,Zion Williamson,0.052041
5,Tyrese Haliburton,0.049542
4,Ja Morant,0.049501
3,DeAndre Jordan,0.039445
2,Domantas Sabonis,0.038318
1,Trevor Hudgins,0.038031
0,Trae Young,0.033949


In [ ]:
# https://stats.stackexchange.com/questions/261704/training-a-neural-network-for-regression-always-predicts-the-mean

# Predict pct